In [522]:
# Base
import time
import datetime

# Data Manipulation
import pandas as pd
import polars as pl

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
#from adjustText import adjust_text

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Scripts
from Scripts.fetch_utils import fetch_league
from Scripts.analytic_utils import *
from Scripts.scrape_player_stats import *
from Scripts.scrape_team_stats import *
from Scripts.luck_index import *
from Scripts.tidbit_utils import *
from Scripts.simulation_utils import *
#from Scripts.lineup_projections import *

#Fantasy_Pros_Projection_Data = 

# Constants
espn_id = 'AEB61FldCpFLVPDUlhuAq0HiJPhiRqGeqVZoSv1gqxveRGB%2BLzGTtEXE3FIvrPK3qoGrOjXE%2FoKqJADgjzYTsJva22%2BtxhsiEQOiLaBNuh%2FoLLfyY4pewmwNQvDnQ6paTcGaOfktKbZidsvEUc061%2FFYnMxMJ596%2F9HrQQCsbH575%2Bp6bi1Vy80GEVtgjfmp03dSSdjsYoxDMVn403%2BI8rB1HkfFAGcnbbXJq9wVjN6E1xglOkDprydZk5HrX%2FOr7QFt1DBNZHVdv52RO6jEEDTUBGNCYgQgzzTTsl%2FaVw8CgNfilAnku6DzS%2BZFoGhhtw8%3D'
s_id = '{796FF49A-04B4-481F-9132-1F91C9585A05}'

lg_vars = {
    "Winfield_Football": {
        "ID": 521152,
        "ESPN_S2": "AECd6kezzt5c3URGK%2FEHxqSyREcJWZX7JUHahtPHI8SV8KK13PkZG98FJYsIQ%2FFTiA5YXhcfcaQ7bGRGZZzzgz7BQSci5bFMfx1xPTCM95tyCvx%2BBJgIACFkxz%2B5J3uz0Ez87c1XWV%2FuQB%2FjgXXBRvAZaMIBaSnfJ842nwUpjS4x6U6r6FNi900ORFBxtdKU%2FQ6Bvr6UVZKerid0TdatKkFq0IrRYlFr3npOSWsv5bzIV9uh8HidTJlYx%2Bf3DYqZABpGEZfLeuJyymEQ%2BJjXGbaOAjZhOP%2FRrrqpS0xvYbFiwET2MCEmraBsiJcqWR6VaIM%3D",
        "SWID": "{DC47642F-1CD6-4451-9C5B-08FFE2844F9E}",
        "start": 2016,
        "end": 2025
    },
    "Weenieless_Wanderers": {
        "ID": 537814,
        "ESPN_S2": espn_id,
        "SWID": s_id,
        "start": 2017,
        "end": 2025
    },
    "GOP_Degenerates": {
        "ID": 1727104,
        "ESPN_S2": espn_id,
        "SWID": s_id,
        "start": 2023,
        "end": 2025
    },
    "Knights_FFL": {
        "ID": 1992637246,
        "ESPN_S2": espn_id,
        "SWID": s_id,
        "start": 2022,
        "end": 2025
    },
    "12 Dudes one Cup": {
        "ID": 1895076862,
        "ESPN_S2": "AECd6kezzt5c3URGK%2FEHxqSyREcJWZX7JUHahtPHI8SV8KK13PkZG98FJYsIQ%2FFTiA5YXhcfcaQ7bGRGZZzzgz7BQSci5bFMfx1xPTCM95tyCvx%2BBJgIACFkxz%2B5J3uz0Ez87c1XWV%2FuQB%2FjgXXBRvAZaMIBaSnfJ842nwUpjS4x6U6r6FNi900ORFBxtdKU%2FQ6Bvr6UVZKerid0TdatKkFq0IrRYlFr3npOSWsv5bzIV9uh8HidTJlYx%2Bf3DYqZABpGEZfLeuJyymEQ%2BJjXGbaOAjZhOP%2FRrrqpS0xvYbFiwET2MCEmraBsiJcqWR6VaIM%3D",
        "SWID": "{DC47642F-1CD6-4451-9C5B-08FFE2844F9E}",
        "start": 2024,
        "end": 2025
    },
    "Big Red Fantasy Football": {
        "ID": 820145,
        "ESPN_S2": "AEBEMc4%2BdfLmeqLeujWJ5vXngkv7UEIRNlCK8we%2BOe%2FPqBZbSlGGHINN%2BFue6%2F1%2BAYun1N%2FFqJTDmim%2FY1N1yZj4PatflCQdsSqvnLUx7l%2FdabKDBTJArnWT8PA9IhKdJAAsZgwC%2FlOKx8mJ7gMp11ppccZF6ycJ8%2FSIeKe65LimLfUERkw7cmRhIaXuFlqTb8qQMcMq9L3cjJv7vnBXa%2F2WD3liCL0Wn6ZmzsCRcTb8VALiRUeIHbgBFL4LET7xFUgxFHNYXmzPUObhwTcKGFQ%2FTGVDNUjHXpR%2BC%2BU2rVnm%2Fw%3D%3D",
        "SWID": "{69874D0A-9596-44FA-A011-A1F2C03C5815}",
        "start": 2024,
        "end": 2025
    },
    "John_PC_League": {
        "ID": 492342148,
        "ESPN_S2": "AEBSqnPhjCKC098rTKjRwAUfW0YeVQHec3NDLylTlZh0Jdnoo9EPsrSpkJiGYDWLrK0qpMbFBuJam6WCfVpdkQIDMfo%2Fby%2FHEbXsz%2BqEAY1A%2Fi9%2B3iO8gY52%2B3JygwBI1kHxqAz2MBGObrBfRykfiKgoafS1rHIho%2FBVXh3cA5UknNbKEZHyZPFT%2BL6ucnenB2zUyVII1eCljiNJYv4QfjLK75q3NGq0zD6acd5tuRot9lS9LK8UYbQkqihTO%2Fy4sCgep2NOBfK7FwKlh0B%2BlIKF",
        "SWID": "{8E5680E0-65DA-4726-B06D-8EA89411AF1E}",
        "start": 2024,
        "end": 2025
    },
    "John_ATL_League": {
        "ID": 727487122,
        "ESPN_S2": "AEBSqnPhjCKC098rTKjRwAUfW0YeVQHec3NDLylTlZh0Jdnoo9EPsrSpkJiGYDWLrK0qpMbFBuJam6WCfVpdkQIDMfo%2Fby%2FHEbXsz%2BqEAY1A%2Fi9%2B3iO8gY52%2B3JygwBI1kHxqAz2MBGObrBfRykfiKgoafS1rHIho%2FBVXh3cA5UknNbKEZHyZPFT%2BL6ucnenB2zUyVII1eCljiNJYv4QfjLK75q3NGq0zD6acd5tuRot9lS9LK8UYbQkqihTO%2Fy4sCgep2NOBfK7FwKlh0B%2BlIKF",
        "SWID": "{8E5680E0-65DA-4726-B06D-8EA89411AF1E}",
        "start": 2024,
        "end": 2025
    }
}

## 1) Set League ##

In [523]:
select_league = "Weenieless_Wanderers"
# Tommy Leagues: Winfield_Football, Knights_FFL, GOP_Degenerates, Weenieless_Wanderers
# Will Leagues: 12 Dudes one Cup
# Cooleen Leagues: Big Red Fantasy Football
# Baizer Leagues: John_PC_League, John_ATL_League

league = fetch_league(
        league_id=lg_vars[select_league]['ID'],
        year=lg_vars[select_league]['end'],
        swid=lg_vars[select_league]['SWID'],
        espn_s2=lg_vars[select_league]['ESPN_S2']
    )

BUILDING 2025 Season For Weenieless Wanderers 


### Current Analysis

#### Current Lineup

In [524]:
lineups = get_ply_stats_by_matchup(league_id=lg_vars[select_league]['ID'],
                year=2025,
                swid=lg_vars[select_league]['SWID'],
                espn_s2=lg_vars[select_league]['ESPN_S2'])

free_agents = build_fa_market(league=league)

curr_week = league.current_week

lineups = pd.concat([lineups, free_agents])

lineups.fillna(0, inplace=True)

lineups = lineups.drop_duplicates(subset=['week', 'player_name'])

BUILDING 2025 Season For Weenieless Wanderers 


In [525]:
def change_col_prefix(df, old_pfix, new_pfix):

    df = df
    df.columns = df.columns.str.replace(f'{old_pfix}', new_pfix, regex=False)
    return df

def impute_columns(df, target_prefix, source_prefix):
    target_cols = [col for col in df.columns if col.startswith(target_prefix)]
    source_cols = [col for col in df.columns if col.startswith(source_prefix)]

    for source_col in source_cols:
        # Define the corresponding target column name
        target_col = target_prefix + source_col[len(source_prefix):]
        
        # If the target column does not exist, create it by copying the values from the source column
        if target_col not in df.columns:
            df[target_col] = df[source_col]
        
        # If the target column exists, impute missing values from the source column
        elif source_col in df.columns:
            df[target_col] = df[target_col].fillna(df[source_col])
    
    return df

def create_mean_cols(df, target_prefix, source_prefix, mean_prefix='MEAN_'):
    target_cols = [col for col in df.columns if col.startswith(target_prefix)]
    source_cols = [col for col in df.columns if col.startswith(source_prefix)]

    for source_col in source_cols:
        target_col = target_prefix + source_col[len(source_prefix):]
        mean_col = mean_prefix + source_col[len(source_prefix):]

        if target_col in df.columns and source_col in df.columns:
            df[mean_col] = df[[target_col, source_col]].mean(axis=1)

    df = df[['week', 'player_name', 'primaryPosition','player_active_status']  + list(df.filter(like='MEAN_').columns)]

    return df

def clean_pinny(pinny_path="Data/Projections/Pinnacle/Season/Pinnacle_Props_Week_All.parquet"):

    # Constants
    prop_to_stat={
        "Touchdowns": 'rushingTouchdowns',
        "Rushing Yards": "rushingYards",
        "Rush Attempts": 'rushingAttempts',

        "Receiving Yards": "receivingYards",
        "Receptions": "receivingReceptions",

        "Touchdown Passes": "passingTouchdowns",
        "Pass Completions": "passingCompletions",
        "Pass Attempts": "passingAttempts",
        "Passing Yards": "passingYards",
        "Interceptions": "passingInterceptions"
    }

    name_changes={
        # Pinny -> ESPN #
        "Tre Harris": "Tre' Harris",
        "Marvin Mims": "Marvin Mims Jr.",
        "Travis Etienne": "Travis Etienne Jr.",
        "Aaron Jones": "Aaron Jones Sr.",
        "Kyle Pitts": "Kyle Pitts Sr.",
        "Calvin Austin": "Calvin Austin III",
        "Ollie Gordon":"Ollie Gordon II",
        "Marvin Harrison": "Marvin Harrison Jr.",
        "Kyle Pitts": "Kyle Pitts Sr.",
        "Marvin Mims": "Marvin Mims Jr.",
        "Travis Etienne": "Travis Etienne Jr.",
        "Aaron Jones": "Aaron Jones Sr."
    }

    # Load
    raw=pd.read_parquet(pinny_path)

    # Get Stats
    #raw.replace({"PropType": prop_to_stat}, inplace=True)

    # Filter
    #raw = raw[raw['PropType'].isin(list(prop_to_stat.values()))]

    # Clean Names
    raw.replace({"player_name": name_changes}, inplace=True)

    # Replace NaN in Value with ImpNoVig
    #raw['Value'] = raw['Value'].fillna(raw['ImpNoVig'])

    def adjust_value(df):
        pivoted_df = df.pivot_table(
            index=['officialDate', 'week', 'Away', 'Home', 'Player', 'PropType', 'Value', 'BetTimeStamp'],
            columns='OverUnder',
            values=['Price', 'Implied', 'ImpNoVig'],
            aggfunc='first'  # Use 'first' in case there are duplicates
        ).reset_index()

        # Flatten the column names and add OverUnder values as suffixes
        pivoted_df.columns.name = None
        new_columns = []

        for col in pivoted_df.columns:
            if col[0] in ['Price', 'Implied', 'ImpNoVig']:
                new_columns.append(f"{col[0]}_{col[1]}")
            else:
                new_columns.append(col[0])

        pivoted_df.columns = new_columns

        # Create Adjusted Values From Juice
        pivoted_df["Juice"] = pivoted_df['Implied_Over'] + pivoted_df['Implied_Under']
        pivoted_df["Over_Juice"] = (1 / pivoted_df['Implied_Over'] - 1)
        pivoted_df["Under_Juice"] = (1 / pivoted_df['Implied_Under'] - 1)
        pivoted_df["Juice_Diff"] = pivoted_df["Under_Juice"] - pivoted_df["Over_Juice"]
        pivoted_df["AdjValue"] = pivoted_df["Value"] + (pivoted_df["Juice_Diff"] * pivoted_df["Value"] * 0.5)

        return pivoted_df
    
    #adjusted = adjust_value(raw)


    #adjusted = adjusted[['week', 'Player', 'PropType', 'AdjValue']]
    #adjusted.columns = ['week', 'player_name', 'statType', 'statValue']

    # Pivot
    #clean = adjusted.pivot_table(index=['week','player_name'], columns='statType', values='statValue', aggfunc='mean').reset_index()

    

    # Split Touchdowns by Usage (Rushing/Receiving)
    #if 'rushingTouchdowns' in clean.columns:
    #    clean['receivingTouchdowns'] = clean['rushingTouchdowns'] * (clean['receivingYards'] / (clean['receivingYards'] + clean['rushingYards']))
    #    clean['rushingTouchdowns'] = clean['rushingTouchdowns'] - clean['receivingTouchdowns']

    #clean.columns = ['week', 'player_name'] + ['proj_' + str(col) for col in clean.columns[2:]]
    #clean = clean[['week', 'player_name'] + list(clean.columns[2:])]

    # Flatten If Needed
    #clean.columns.name = None
    #clean.columns = [col if col is not None else 'StatValue' for col in clean.columns]

    #final = proj_to_score(proj_df=clean, col_pfix="PINNY")

    return raw

def clean_bol(bol_path = "Data/Projections/BetOnline/Season/BetOnline_AllProps.parquet"):

    raw = pd.read_parquet(bol_path).drop(columns=['position', 'team'])

    name_changes={
        # BOL -> ESPN #
        "Tre Harris": "Tre' Harris",
        "Kyle Pitts": "Kyle Pitts Sr.",
        "Deebo Samuel Sr.":"Deebo Samuel",
        "Cameron Ward":"Cam Ward",
        "Marquise Brown":"Hollywood Brown",
        "Ray-Ray McCloud": "Ray-Ray McCloud III"
    }

    if 'proj_defensiveTotalTackles' in raw.columns:
        raw['proj_defensiveSoloTackles'] = raw['proj_defensiveTotalTackles'] * 0.75
        raw['proj_defensiveAssistedTackles'] = raw['proj_defensiveTotalTackles'] * 0.5

    raw.replace({"player_name": name_changes}, inplace=True)

    return raw

def get_match_details(df1, df2, keys, check_col2, min_wk, tbl_lab):

    # Only Check Weeks that Exist in Data w/ Projected Stats
    df1 = df1[(df1['week'] == min_wk) & (~df1['primaryPosition'].isin(['D/ST', 'K', 'DL', 'DE', 'LB', 'NT', 'CB', 'S', 'DT', 'DB', 'OLB']))]
    df1 = df1[df1.filter(like='MEAN_').sum(axis=1) > 0]
    df1 = df1[df1['player_active_status'] == 'active']

    # Option 1 - Count unmatched values
    merged_df = pd.merge(df1, df2, on=keys, how='left')
    unmatched_from_df2 = merged_df[check_col2].isnull().sum()
    
    # If > 0, print more information
    if unmatched_from_df2 > 0:
        print(f'Unmatched from {tbl_lab}: {unmatched_from_df2}')

        merged_df = pd.merge(df1, df2, on=keys, how='left', indicator=True)
        unmatched_rows = merged_df[merged_df['_merge'] != 'both']
        unmatched_count = unmatched_rows.shape[0]
        print(unmatched_rows[['week', 'player_name']])

        # Option 3 - Grouping To get Count
        unmatched_count = merged_df['_merge'].value_counts()
        print(unmatched_count)
        print(" ")
    elif unmatched_from_df2 == 0:
        print(f"All Rows in {tbl_lab} Match")
        print(" ")

def compute_weighted_stats(df, stats_list, weights_dict):
    """
    Compute weighted stats for the given statistics in the DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the stats.
    - stats_list (list): A list of stat prefixes to compute.
    - weights_dict (dict): A nested dictionary where keys are stat prefixes and values are dictionaries
                           of sources and their corresponding weights.

    Returns:
    - pd.DataFrame: The updated DataFrame with the new weighted stat columns.
    """
    for stat in stats_list:
        new_column_name = f'TRUE_{stat}'
        df[new_column_name] = 0  # Initialize the new column

        # Check if the stat has specific weights defined
        if stat in weights_dict:
            for source, weight in weights_dict[stat].items():
                # Construct the column name based on the source and stat
                col_name = f"{source}_{stat}"
                if col_name in df.columns:
                    # Add the weighted contribution to the new column
                    df[new_column_name] += df[col_name] * weight
        else:
            # If no specific weights are defined for the stat, use default weights
            for source, weight in weights_dict['default'].items():
                col_name = f"{source}_{stat}"
                if col_name in df.columns:
                    df[new_column_name] += df[col_name] * weight
    
    return df

scores_df = build_scoring_table(league=league)

def proj_to_score(proj_df, col_pfix_list=['ESPN', 'FP', 'MEAN', 'PINNY', 'BOL', 'TRUE'], s_df=scores_df):

    # Update scores in s_df for D/ST positions and specific conditions
    if 1727104 in list(proj_df['league_id'].unique()):
        dp_df = proj_df[proj_df['primaryPosition'].isin(['DL', 'DE', 'LB', 'NT', 'CB', 'S', 'DT', 'DB', 'OLB'])]
        normal_df = proj_df[~proj_df['primaryPosition'].isin(['DL', 'DE', 'LB', 'NT', 'CB', 'S', 'DT', 'DB', 'OLB'])]

        s_df.loc[s_df['id'] == 99, 'points'] = 1
        s_df.loc[s_df['id'] == 109, 'points'] = 0
        s_df.loc[s_df['id'] == 112, 'points'] = 0
        s_df.loc[s_df['id'] == 113, 'points'] = 0
        s_df.loc[s_df['id'] == 95, 'points'] = 1
        s_df.loc[s_df['id'] == 97, 'points'] = 1

        # Iterate over each prefix in the prefix list
        for col_pfix in col_pfix_list:
            # Initialize the score column to 0
            normal_df[f'{col_pfix}_Points'] = 0.0

            # Iterate over each score row in s_df
            for _, score_row in s_df.iterrows():
                col_name = f"{col_pfix}_{score_row['colName']}"

                # If the corresponding stat column exists in proj_df, calculate the weighted score
                if col_name in normal_df.columns:
                    normal_df[f'{col_pfix}_Points'] += normal_df[col_name] * score_row['points']


        # Handle DP 
        s_df.loc[s_df['id'] == 95, 'points'] = 12
        s_df.loc[s_df['id'] == 97, 'points'] = 2
        s_df.loc[s_df['id'] == 99, 'points'] = 10
        s_df.loc[s_df['id'] == 112, 'points'] = 5
        s_df.loc[s_df['id'] == 113, 'points'] = 5
        s_df.loc[s_df['id'] == 109, 'points'] = 1

        # Iterate over each prefix in the prefix list
        for col_pfix in col_pfix_list:
            # Initialize the score column to 0
            dp_df[f'{col_pfix}_Points'] = 0.0

            # Iterate over each score row in s_df
            for _, score_row in s_df.iterrows():
                col_name = f"{col_pfix}_{score_row['colName']}"

                # If the corresponding stat column exists in proj_df, calculate the weighted score
                if col_name in dp_df.columns:
                    dp_df[f'{col_pfix}_Points'] += dp_df[col_name] * score_row['points']

        proj_df = pd.concat([normal_df, dp_df])

    else:
        # Iterate over each prefix in the prefix list
        for col_pfix in col_pfix_list:
            # Initialize the score column to 0
            proj_df[f'{col_pfix}_Points'] = 0.0

            # Iterate over each score row in s_df
            for _, score_row in s_df.iterrows():
                col_name = f"{col_pfix}_{score_row['colName']}"

                # If the corresponding stat column exists in proj_df, calculate the weighted score
                if col_name in proj_df.columns:
                    proj_df[f'{col_pfix}_Points'] += proj_df[col_name] * score_row['points']
    
    return proj_df

def clean_lineups(df=lineups):

    # Get Base of Projections (player_name, week, team, etc.)
    base_cols = ['league_id','year','week', 'team_owner', 'team_name', 'team_division', 'player_name', 'player_id', 'slotPosition', 'primaryPosition', 'eligiblePositions', 'pro_team', 'current_team_id' ,'player_position' ,'player_active_status', 'points', 'projPoints']
    actual_scoring_cols = scores_df['colName'].to_list()
    base = df[base_cols + actual_scoring_cols]

    # Constants

    ## Defensive Positions
    d_pos = ['DL', 'DE', 'LB', 'NT', 'CB', 'S', 'DT', 'DB', 'OLB']

    ## Duplicate Error
    fix_list = ['ESPN_rushingYards', 'ESPN_receivingYards', 'ESPN_passingYards']

    # 1) Combine ESPN and Fantasy Pros Data

    ## a) Build ESPN From Raw Data
    espn_proj = df[['week', 'player_name', 'primaryPosition', 'player_active_status']  + list(df.filter(like='proj_').columns)]
    espn_proj = change_col_prefix(df=espn_proj, old_pfix="proj", new_pfix="ESPN")

    ## b) Build Fantasy Pros From Scrape
    fp_proj = pd.read_parquet("Data/Projections/FantasyPros/FantasyPros_Projections_Week_All.parquet").drop(columns=['STD_FantasyPoints', 'TimeStamp'])
    fp_proj = change_col_prefix(df=fp_proj, old_pfix="proj", new_pfix="FP")

    ## c) Combine ESPN and FP
    trans1_df = espn_proj.merge(fp_proj, how='left', on=['week', 'player_name'])
    get_match_details(df1=espn_proj, df2=fp_proj, keys=["week", "player_name"], check_col2="FP_rushingTouchdowns", min_wk=curr_week, tbl_lab="FantasyPros Table")

    ## d) Correct Error Where ESPN Duplicates Yards
    print("=============================== Correcting ESPN Doubled Yard Projections ===============================")
    print(" ")
    for column in fix_list:
        FP_Col = column.replace("ESPN_", "FP_")
        # Apply with a conditional print statement
        trans1_df[column] = trans1_df.apply(
            lambda row: (
                # Check if the condition is true
                print(f"Player: {row['player_name']}, Week: {row['week']}, "
                      f"Original {column}: {row[column]}, "
                      f"New {column}: {row[column] / 2}") or row[column] / 2
                if (row[column] > (row[FP_Col] * 1.75)) and (row[column] > 40) else row[column]
            ),
            axis=1
        )

    print(" ")
    print("======================================== End Doubled Correction ========================================")
    print(" ")

    ## e) Impute FP with ESPN + Create Means
    trans1_df = impute_columns(trans1_df, target_prefix='FP_', source_prefix='ESPN_')
    mean_df = create_mean_cols(trans1_df, target_prefix='FP_', source_prefix='ESPN_')

    ## f) Retain New ESPN Values For Join With Books + Add To Base
    base = base.merge(trans1_df, on=['week', 'player_name', 'primaryPosition','player_active_status'], how='left')

    ## f) Create Dataframe of Means For Imputing Sportsbook Data
    mean_df = create_mean_cols(trans1_df, target_prefix='FP_', source_prefix='ESPN_')
    

    # 2) Combine Pinnacle Data With ESPN and Impute
    ## a) Clean Pinnacle Data
    pinny_proj = clean_pinny() #pd.read_parquet("Data/Projections/Pinnacle/Season/Pinnacle_Props_Week_All.parquet")
    pinny_proj = change_col_prefix(df=pinny_proj, old_pfix="proj", new_pfix="PINNY")

    ## b) Impute Missing Data From ESPN
    trans2_df = mean_df.merge(pinny_proj, on=["week", "player_name"], how='left')
    get_match_details(df1=mean_df, df2=pinny_proj, keys=["week", "player_name"], check_col2="PINNY_receivingYards", min_wk=curr_week-1, tbl_lab="Pinnacle Sportsbook Table")
    trans2_df = impute_columns(trans2_df, target_prefix='PINNY_', source_prefix="MEAN_")
    

    ## c) Slim Columns To Only Pinnacle Data
    trans2_df = trans2_df[['week', 'player_name', 'primaryPosition','player_active_status'] + list(trans2_df.filter(like='PINNY').columns)]
    ## d) Join Slim Transformation Back To Base
    base = base.merge(trans2_df, on=['week', 'player_name', 'primaryPosition','player_active_status'], how='left')


    # 3) Combine BetOnline Data With ESPN and Impute
    bol_proj = clean_bol()
    bol_proj = change_col_prefix(df=bol_proj, old_pfix="proj", new_pfix="BOL")

    ## b) Impute Missing Data From ESPN
    trans3_df = mean_df.merge(bol_proj, on=["week", "player_name"], how='left')
    get_match_details(df1=mean_df, df2=bol_proj, keys=["week", "player_name"], check_col2="BOL_rushingTouchdowns", min_wk=curr_week-1, tbl_lab="BetOnline Sportsbook Table")
    trans3_df = impute_columns(trans3_df, target_prefix='BOL_', source_prefix="MEAN_")

    ## c) Slim Columns To Only BOL Data
    trans3_df = trans3_df[['week', 'player_name', 'primaryPosition','player_active_status'] + list(trans3_df.filter(like='BOL_').columns)]

    ## d) Join Slim Transformation Back To Base
    base = base.merge(trans3_df, on=['week', 'player_name', 'primaryPosition','player_active_status'], how='left')

    ## Clean Missing COlumns
    base = impute_columns(base, target_prefix='PINNY_', source_prefix='MEAN_')
    base = impute_columns(base, target_prefix='BOL_', source_prefix='MEAN_')


    ## 5) Create Aggregate Columns For Each Projection Type (Manual Weights)
    weights_dict = {
        'passingYards': {'ESPN': 0.1,'FP': 0.7,'PINNY': 0.1,'BOL': 0.1},
        'passingTouchdowns': {'ESPN': 0.1,'FP': 0.1,'PINNY': 0.4,'BOL': 0.4},
        'rushingYards': {'ESPN': 0.2,'FP': 0.3,'PINNY': 0.25,'BOL': 0.25},
        'receivingYards': {'ESPN': 0.2,'FP': 0.3,'PINNY': 0.25,'BOL': 0.25},
        'default': {'ESPN': 0.2,'FP': 0.3,'PINNY': 0.25,'BOL': 0.25}
    }

    final = compute_weighted_stats(df=base, stats_list=actual_scoring_cols, weights_dict=weights_dict)

    ## 6) Build Score Column
    final = proj_to_score(final)

    # a) Adjust For Extra ESPN Stats
    final['adjustment'] = final['projPoints'] - final['ESPN_Points']
    for i in ['ESPN', 'FP', 'MEAN', 'PINNY', 'BOL', 'TRUE']:
        final[f'{i}_Points'] = final['adjustment'] + final[f'{i}_Points']


    ## 7) Build Position Rank Columns
    for i in ['ESPN', 'FP', 'MEAN', 'PINNY', 'BOL', 'TRUE']:
        final[f"{i}_PosRank"] = final.groupby(['week', 'primaryPosition'])[f'{i}_Points'].rank(ascending=False, method='dense')

    # Actual
    final['PosRank'] = final.groupby(['week', 'primaryPosition'])['points'].rank(ascending=False, method='dense')

    return final

LINEUPS = clean_lineups()


All Rows in FantasyPros Table Match
 
=============================== Correcting ESPN Doubled Yard Projections ===============================
 
Player: Jayden Daniels, Week: 3, Original ESPN_rushingYards: 100.2502262, New ESPN_rushingYards: 50.1251131
Player: Brock Bowers, Week: 2, Original ESPN_receivingYards: 145.6468043, New ESPN_receivingYards: 72.82340215
Player: Jaylen Waddle, Week: 3, Original ESPN_receivingYards: 130.4050653, New ESPN_receivingYards: 65.20253265
Player: Christian Kirk, Week: 3, Original ESPN_receivingYards: 93.07232176, New ESPN_receivingYards: 46.53616088
Player: Brian Thomas Jr., Week: 3, Original ESPN_receivingYards: 148.0516801, New ESPN_receivingYards: 74.02584005
Player: Dallas Goedert, Week: 3, Original ESPN_receivingYards: 75.23833339, New ESPN_receivingYards: 37.619166695
Player: Xavier Worthy, Week: 3, Original ESPN_receivingYards: 110.5760834, New ESPN_receivingYards: 55.2880417
Player: Jauan Jennings, Week: 3, Original ESPN_receivingYards: 105.4979

In [526]:
#pl.read_csv("Data/Projections/Pinnacle/Season/Pinnacle_Props_Week_All.csv")
#pl.read_parquet("Data/Projections/Pinnacle/Season/Pinnacle_Props_Week_All.parquet")
#pl.read_csv("Data/Projections/Pinnacle/Season/Pinnacle_Props_Week_All.csv")\
#    .with_columns([
#        pl.col('officialDate').cast(pl.Date),
#        pl.col('BetTimeStamp').cast(pl.Datetime)
#    ])\
#    .write_parquet("Data/Projections/Pinnacle/Season/Pinnacle_Props_Week_All.parquet")

### Algorithims

In [527]:
import statsmodels.api as sm
def get_weight_by_stat(df, stat):

    train_df = df[(df['week'] < league.current_week) & (df[f"{stat}"] > 0) & (df["player_active_status"] == 'active')]
    test_df = df[df['week'] == league.current_week]

    ## Pull Features

    # Projected Stats
    proj_cols = train_df.filter(like=f'_{stat}').columns
    # Volume Stats
    if "passing" in stat:
        vol_cols = train_df.filter(regex='_passingCompletions').columns
        pos_filter = ['QB']
    elif "rushing" in stat:
        vol_cols = train_df.filter(regex='_rushingAttempts').columns
        pos_filter = ['RB', 'WR']
    elif "receiving" in stat:
        vol_cols = train_df.filter(regex='_receivingReceptions').columns
        pos_filter = ['RB', 'WR', 'TE']
    else:
        vol_cols = []

    train_df = train_df[train_df['primaryPosition'].isin(pos_filter)].reset_index()


    feature_cols = list(proj_cols) + list(vol_cols)
    
    # Features
    X = train_df[feature_cols]
    X = sm.add_constant(X)

    # Target
    y = train_df[f'{stat}']

    # Build Model
    model = sm.OLS(y, X).fit()
    weights = model.params

    # Create Projection
    test_df = test_df[(test_df['primaryPosition'].isin(pos_filter)) & (test_df['player_active_status'] == 'active')]
    test_df[f'PRED_{stat}'] = weights['const'] + sum(test_df[col] * weights[col] for col in feature_cols)
    clean = test_df[['week', 'team_name', 'team_owner', 'player_name', 'primaryPosition', 'player_active_status', f'{stat}'] + list(test_df.filter(like=f'_{stat}').columns)]

    return clean.sort_values(by=[f'PRED_{stat}'], ascending=False)

#get_weight_by_stat(df=LINEUPS, stat="receivingYards")

In [528]:
def peek_proj_stats(stat, wk, pos_filter = ['QB', 'RB', 'WR', 'TE']):
    # Build Dataframe
    FULL=LINEUPS[(LINEUPS['week'] == wk) & LINEUPS['primaryPosition'].isin(pos_filter) & (LINEUPS['BOL_Points'] != LINEUPS['TRUE_Points'])]
    SLIM = FULL[['week', 'player_name', 'primaryPosition']  + list(FULL.filter(like=f'{stat}').columns)]
    SLIM = SLIM[SLIM.filter(like=f'{stat}').sum(axis=1) > 0]

    # Create True_Points
    if stat == 'defensiveInterceptions' and "D/ST" not in pos_filter:
         pt_val = 12
    elif stat == 'defensiveSacks' and "D/ST" not in pos_filter:
         pt_val = 10
    elif stat == 'defensiveSoloTackles' and "D/ST" not in pos_filter:
         pt_val = 2
    elif stat == 'defensiveAssistedTackles' and "D/ST" not in pos_filter:
         pt_val = 0.5
    else:
        pt_val = scores_df[scores_df['colName'] == stat]['points'].iloc[0]

        
    SLIM['true_Points'] = SLIM[f'TRUE_{stat}']*pt_val

    return SLIM.sort_values(by=f'true_Points', ascending=False)

peek_proj_stats(stat='passingTouchdowns', wk=curr_week, pos_filter=['QB']).head()

week      player_name primaryPosition  passingTouchdowns  \
350     3    Lamar Jackson              QB                0.0   
440     3   Baker Mayfield              QB                0.0   
474     3       Josh Allen              QB                0.0   
409     3     Dak Prescott              QB                0.0   
351     3  Patrick Mahomes              QB                0.0   

     ESPN_passingTouchdowns  FP_passingTouchdowns  MEAN_passingTouchdowns  \
350                1.964708                   2.0                1.982354   
440                2.177204                   1.8                1.988602   
474                1.809351                   1.8                1.804675   
409                1.797629                   1.8                1.798815   
351                1.733539                   1.6                1.666770   

     PINNY_passingTouchdowns  BOL_passingTouchdowns  TRUE_passingTouchdowns  \
350                 1.982354                 2.2425                2.086412   
440                 1.988602                 1.9650                1.979161   
474                 1.928250                 1.9200                1.900235   
409                 1.798815                 1.8750                1.829289   
351                 1.666770                 1.9650                1.786062   

     true_Points  
350     8.345650  
440     7.916645  
474     7.600940  
409     7.317155  
351     7.144247

In [535]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
def create_ESPN_combo(stat, visualize = True, pos_filter = None):

    # Build Combination Table
    df = LINEUPS
    df['Points'] = df['points']

    ## Filter Where Stat != ESPN & != AVG(ESPN, FP)
    df = df[(df[f'PINNY_{stat}'] != ((df[f'ESPN_{stat}'] + df[f'FP_{stat}'])/2)) & (df[f'BOL_{stat}'] != ((df[f'ESPN_{stat}'] + df[f'FP_{stat}'])/2))]

    ## Filter For Completed Weeks
    df = df[(df['week'] < curr_week) & (~df[f'ESPN_{stat}'].isna())]
    df[f'{stat}'].fillna(0, inplace=True)

    # DF Filter Rows Where Projection is 0 in Pinny
    df = df[df[f'PINNY_{stat}'] > 0]

    if pos_filter is not None:
        df = df[df['primaryPosition'].isin(pos_filter)]

    # Filter any NAs
    df = df.dropna(subset=[f'PINNY_{stat}', f'FP_{stat}'])

    # Build Linear Regression Model
    X = df[[f'ESPN_{stat}', f'FP_{stat}', f'PINNY_{stat}', f'BOL_{stat}']]
    Y = df[f'{stat}']
    model = LinearRegression()
    model.fit(X, Y)
    df[f'COMBO_{stat}'] = model.predict(X)



    # Build Difference Cols
    #df[f"DIFF_ESPN_{stat}"] = df[f"{stat}"] - df[f'ESPN_{stat}']
    #df[f"DIFF_FP_{stat}"] = df[f"{stat}"] - df[f'FP_{stat}']
    #df[f"DIFF_PINNY_{stat}"] = df[f"{stat}"] - df[f'PINNY_{stat}']

    clean = df[['week', 'player_name', f"{stat}", f'ESPN_{stat}', f'FP_{stat}', f'PINNY_{stat}', f'BOL_{stat}', f'TRUE_{stat}', f'COMBO_{stat}']].sort_values(by=[f"{stat}"], ascending=False)

    if visualize == True:

        label_map = {f"TRUE_{stat}": "Average", f"PINNY_{stat}": "Pinnacle Sportsbook", f"BOL_{stat}": "BetOnline Sportsbook", f"ESPN_{stat}": "ESPN", f"FP_{stat}": "FantasyPros", f"COMBO_{stat}": "Model"}

        df_melted = clean.melt(id_vars=['player_name', f'{stat}'], 
                    value_vars=[f'ESPN_{stat}', f'FP_{stat}', f'PINNY_{stat}', f'BOL_{stat}', f'COMBO_{stat}', f'TRUE_{stat}'],
                    var_name='Projection Source', value_name=f'Projected {stat}')
        
        df_melted.replace({"Projection Source": label_map}, inplace=True)

        r2_scores = {}
        for source in df_melted['Projection Source'].unique():
            subset = df_melted[df_melted['Projection Source'] == source]
            r2 = r2_score(subset[f'{stat}'], subset[f'Projected {stat}'])
            r2_scores[source] = r2


        # Plot using seaborn
        plt.figure(figsize=(12, 7))
        sns.scatterplot(x=f'Projected {stat}', y=f'{stat}', hue='Projection Source', data=df_melted, palette='Set1')


        handles, labels = plt.gca().get_legend_handles_labels()
        new_labels = [f'{label} (R² = {r2_scores[label]:.3f})' for label in labels]
        plt.legend(handles, new_labels, title='Projection Source')

        plt.title(f'{stat} vs Projected {stat} by Projection Source')
        plt.xlabel('Projected')
        plt.ylabel('Actual')
        plt.show()

    elif visualize == False:
        return clean

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

create_ESPN_combo(stat = "receivingReceptions", pos_filter=['WR','TE', 'RB'], visualize=True) #

In [530]:
def check_week(lu, week, own):

    lu['trueDiff'] = lu['TRUE_Points'] - lu['projPoints']

    # Get My Team
    df = lu[(lu['week'] == week) & (lu['team_owner'] == own)][['week', 'team_name', 'player_name', 'slotPosition', 'primaryPosition',
                                                               'points', 'projPoints', 'FP_Points', 'PINNY_Points', 'BOL_Points', 'TRUE_Points', 'trueDiff',
                                                               'PosRank', 'ESPN_PosRank', 'FP_PosRank', 'PINNY_PosRank', 'BOL_PosRank', 'TRUE_PosRank']]
    
    df = df.rename(columns={
        'team_name': 'team',
        'player_name': 'player',
        'slotPosition': 'rosPos',
        'primaryPosition': 'primPos',
        'points': 'Actual_PTS',
        'projPoints': 'ESPN_PTS',
        'FP_Points': 'FP_PTS',
        'PINNY_Points': 'PINNY_PTS',
        'BOL_Points': 'BOL_PTS',
        'TRUE_Points': 'TRUE_PTS',
        'trueDiff': 'DIFF_PTS',
        'PosRank': 'Actual',
        'ESPN_PosRank': 'ESPN',
        'FP_PosRank': 'FP',
        'PINNY_PosRank': 'PINNY',
        'BOL_PosRank': 'BOL',
        'TRUE_PosRank': 'TRUE',
    })

    # Order
    bench = pd.DataFrame([{
        'week': week,
        'team': df['team'].iloc[0],
        'player': 'Total',
        'rosPos': 'Starting Lineup',
        'primPos': '',
        'Actual_PTS': df[~df['rosPos'].isin(['BE', 'IR'])]['Actual_PTS'].sum(),
        'ESPN_PTS': df[~df['rosPos'].isin(['BE', 'IR'])]['ESPN_PTS'].sum(),
        'FP_PTS': df[~df['rosPos'].isin(['BE', 'IR'])]['FP_PTS'].sum(),
        'PINNY_PTS': df[~df['rosPos'].isin(['BE', 'IR'])]['PINNY_PTS'].sum(),
        'BOL_PTS': df[~df['rosPos'].isin(['BE', 'IR'])]['BOL_PTS'].sum(),
        'TRUE_PTS': df[~df['rosPos'].isin(['BE', 'IR'])]['TRUE_PTS'].sum(),
        'DIFF_PTS': df[~df['rosPos'].isin(['BE', 'IR'])]['DIFF_PTS'].sum(),
        'Actual': '',
        'ESPN': '',
        'FP': '',
        'PINNY': '',
        'BOL': '',
        'TRUE': ''
        
    },
    {
        'week': week,
        'team': df['team'].iloc[0],
        'player': 'Total',
        'rosPos': 'Bench',
        'primPos': '',
        'Actual_PTS': df[df['rosPos'].isin(['BE', 'IR'])]['Actual_PTS'].sum(),
        'ESPN_PTS': df[df['rosPos'].isin(['BE', 'IR'])]['ESPN_PTS'].sum(),
        'FP_PTS': df[df['rosPos'].isin(['BE', 'IR'])]['FP_PTS'].sum(),
        'PINNY_PTS': df[df['rosPos'].isin(['BE', 'IR'])]['PINNY_PTS'].sum(),
        'BOL_PTS': df[df['rosPos'].isin(['BE', 'IR'])]['BOL_PTS'].sum(),
        'TRUE_PTS': df[df['rosPos'].isin(['BE', 'IR'])]['TRUE_PTS'].sum(),
        'DIFF_PTS': df[df['rosPos'].isin(['BE', 'IR'])]['DIFF_PTS'].sum(),
        'Actual': '',
        'ESPN': '',
        'FP': '',
        'PINNY': '',
        'BOL': '',
        'TRUE': ''
        
    }])

    # Append the summary row to the DataFrame
    df = pd.concat([df, bench], ignore_index=True)

    pos_order = ['QB', 'RB', 'WR', 'TE', 'RB/WR/TE', 'OP', 'DP', 'D/ST', 'K', 'Starting Lineup', 'BE', 'Bench', 'IR']
    order_mapping = {val: idx for idx, val in enumerate(pos_order)}
    df = df.sort_values(by=['rosPos', 'TRUE_PTS'], key=lambda x: x.map(order_mapping))

    # Round
    df = df.round(decimals={'FP_PTS': 2, 'PINNY_PTS': 2, 'BOL_PTS': 2, 'TRUE_PTS': 2, 'DIFF_PTS': 3})

    # Drop Actual if Current Week
    if week == curr_week:
        df.drop(['Actual_PTS', 'Actual'], axis=1, inplace=True)

    return df

check_week(lu = LINEUPS, week = curr_week, own='Tommy Winfield')

week     team              player           rosPos primPos  ESPN_PTS  \
6      3  Boobs 2        Dak Prescott               QB      QB     17.61   
0      3  Boobs 2      Saquon Barkley               RB      RB     18.62   
16     3  Boobs 2         Kareem Hunt               RB      RB      9.33   
1      3  Boobs 2        Malik Nabers               WR      WR     19.81   
2      3  Boobs 2    Brian Thomas Jr.               WR      WR     15.33   
11     3  Boobs 2      Kyle Pitts Sr.               TE      TE      9.46   
4      3  Boobs 2       Ladd McConkey         RB/WR/TE      WR     14.61   
17     3  Boobs 2         Romeo Doubs         RB/WR/TE      WR     10.39   
15     3  Boobs 2       Jake Browning               OP      QB     15.48   
14     3  Boobs 2          49ers D/ST             D/ST    D/ST      7.97   
18     3  Boobs 2               Total  Starting Lineup            138.61   
5      3  Boobs 2  TreVeyon Henderson               BE      RB      8.80   
7      3  Boobs 2           RJ Harvey               BE      RB      6.71   
8      3  Boobs 2        Mark Andrews               BE      TE      7.53   
9      3  Boobs 2      Matthew Golden               BE      WR      8.81   
10     3  Boobs 2          Josh Downs               BE      WR      8.68   
12     3  Boobs 2         Jaydon Blue               BE      RB      0.00   
13     3  Boobs 2       Dylan Sampson               BE      RB      4.87   
19     3  Boobs 2               Total            Bench             45.40   
3      3  Boobs 2       Justin Fields               IR      QB      0.00   

    FP_PTS  PINNY_PTS  BOL_PTS  TRUE_PTS  DIFF_PTS  ESPN    FP PINNY   BOL  \
6    17.22      17.42    18.54     17.71     0.102  10.0   9.0   9.0   5.0   
0    18.66      18.64    18.20     18.53    -0.088   5.0   4.0   3.0   5.0   
16    8.20       8.76     9.09      8.79    -0.540  35.0  39.0  38.0  37.0   
1    17.75      18.78    18.42     18.59    -1.222   2.0   5.0   4.0   2.0   
2    13.16      14.25    14.24     14.14    -1.194  13.0  24.0  19.0  18.0   
11    8.93       9.20     9.66      9.29    -0.173  12.0  16.0  14.0  10.0   
4    14.88      14.75    14.84     14.78     0.172  17.0  15.0  15.0  15.0   
17   10.06      10.22     9.08      9.92    -0.469  48.0  52.0  50.0  56.0   
15   14.57      15.02    14.05     14.47    -1.012  18.0  21.0  22.0  18.0   
14    7.97       7.97     7.97      7.97     0.001  13.0  13.0  13.0  13.0   
18  131.41     135.01   134.09    134.19    -4.423                           
5     9.37       9.08     9.46      9.20     0.404  38.0  36.0  37.0  34.0   
7     5.90       6.31     7.24      6.50    -0.211  42.0  47.0  45.0  43.0   
8     9.48       8.50     9.49      8.85     1.318  21.0  11.0  19.0  11.0   
9     9.37       9.09     7.28      8.67    -0.144  55.0  57.0  58.0  68.0   
10    8.58       8.63     9.55      8.85     0.174  58.0  60.0  59.0  53.0   
12    0.00       0.00     0.52      0.13     0.129  67.0  74.0  74.0  71.0   
13    7.16       6.02     6.78      6.32     1.451  53.0  41.0  48.0  47.0   
19   49.86      47.63    52.74     49.13     3.729                           
3     0.00       0.00     2.43      0.61     0.608  34.0  35.0  35.0  34.0   

    TRUE  
6    7.0  
0    4.0  
16  38.0  
1    3.0  
2   19.0  
11  12.0  
4   16.0  
17  52.0  
15  19.0  
14  13.0  
18        
5   37.0  
7   45.0  
8   16.0  
9   60.0  
10  58.0  
12  74.0  
13  47.0  
19        
3   34.0

In [531]:
def get_league_projections(week, lu=LINEUPS):
    df = lu[(lu['week'] == week) & (lu['team_owner'] != 'Free Agent') & (~lu['slotPosition'].isin(['BE', 'IR']))][['week', 'team_owner', 'team_name',
             'points', 'projPoints', 'TRUE_Points']]
    
    df['TRUE_Points'] = df['TRUE_Points'].fillna(df['projPoints'])

    result = df.groupby(['week', 'team_owner', 'team_name'], as_index=False).agg({
        'points': 'sum',
        'projPoints': 'sum',
        'TRUE_Points': 'sum'
        })
    
    result['point_diff'] = result['TRUE_Points'] - result['projPoints']

    return result.sort_values(by='TRUE_Points', ascending=False)

get_league_projections(week = curr_week)

week          team_owner                        team_name  points  \
7     3  Stephen Touchstone             Sandusky Shower Pals     0.0   
4     3         Justin Yoon    Matthew Jordan's Fantasy Team     0.0   
5     3        Logan Walker     Waddle you doing step Burrow     0.0   
3     3  Juan Pablo Urrutia     If it's white, say goodnight     0.0   
1     3        Jeff Wilhelm     Kyren Lacy Collision Center      0.0   
8     3       Tom Schatzman                   L Ron Chubbard     0.0   
2     3          Josh Beall    Nabers think I'm selling dope     0.0   
6     3          Sam Sobell  jp cant take a sh1t w.o AI help     0.0   
9     3      Tommy Winfield                          Boobs 2     0.0   
0     3        Jason Walker            50P APR DODGE HELLCAT     0.0   

   projPoints  TRUE_Points  point_diff  
7      155.18   152.937412   -2.242588  
4      154.61   150.437850   -4.172150  
5      151.31   149.487653   -1.822347  
3      148.62   145.710591   -2.909409  
1      147.74   145.540281   -2.199719  
8      147.82   145.322753   -2.497247  
2      144.47   142.617867   -1.852133  
6      145.74   140.102685   -5.637315  
9      138.61   134.186896   -4.423104  
0      123.70   123.397417   -0.302583

In [532]:
def get_rankings(pos, week, visualize = False, check_fa = False):
    df = LINEUPS[(LINEUPS['primaryPosition'].isin(pos)) & (LINEUPS['week'] == week)]
    df = df[['week', 'primaryPosition','player_name', 'team_owner', 'team_name',
                  'points', 'projPoints', 'TRUE_Points', 'BOL_Points', 'PosRank', 'ESPN_PosRank', 'TRUE_PosRank']]
    if week != curr_week:
        df = df.sort_values(by=['PosRank', 'points'])
    else:
        df = df.drop(columns=['points', 'PosRank']).sort_values(by=['TRUE_Points'], ascending=False)

    if visualize == False:
        if check_fa == True:
            return df[df['team_owner'].isin(['Tommy Winfield', 'John Baizer', 'Free Agent'])]
        else:
            return df
        
get_rankings(pos = ['WR'], week = curr_week, visualize=False, check_fa=True).head(10) #'LB', 'DE', 'S', 'CB', 'DT'


week primaryPosition        player_name      team_owner   team_name  \
404     3              WR       Malik Nabers  Tommy Winfield     Boobs 2   
407     3              WR      Ladd McConkey  Tommy Winfield     Boobs 2   
405     3              WR   Brian Thomas Jr.  Tommy Winfield     Boobs 2   
420     3              WR        Romeo Doubs  Tommy Winfield     Boobs 2   
578     3              WR      Elic Ayomanor      Free Agent  Free Agent   
583     3              WR  Calvin Austin III      Free Agent  Free Agent   
413     3              WR         Josh Downs  Tommy Winfield     Boobs 2   
591     3              WR        Dyami Brown      Free Agent  Free Agent   
412     3              WR     Matthew Golden  Tommy Winfield     Boobs 2   
597     3              WR   Dontayvion Wicks      Free Agent  Free Agent   

     projPoints  TRUE_Points  BOL_Points  ESPN_PosRank  TRUE_PosRank  
404       19.81    18.587900   18.419232           2.0           3.0  
407       14.61    14.781724   14.835815          17.0          16.0  
405       15.33    14.136025   14.239208          13.0          19.0  
420       10.39     9.921348    9.081160          48.0          52.0  
578        9.82     9.608679    8.882693          51.0          55.0  
583        8.73     8.868845    7.998862          56.0          57.0  
413        8.68     8.854054    9.552474          58.0          58.0  
591        8.70     8.711436    9.041071          57.0          59.0  
412        8.81     8.665507    7.284272          55.0          60.0  
597        8.45     8.585306    9.078200          59.0          61.0

In [353]:
grouped_means = LINEUPS[(LINEUPS['player_active_status'] == 'active') & (LINEUPS['week'] == 1)].groupby('primaryPosition').agg({
    'points': 'mean',
    'projPoints': 'mean',
    'ESPN_Points': 'mean',
    'TRUE_Points': 'mean'
}).reset_index().sort_values(by = ['points'], ascending=False)

grouped_means

primaryPosition     points  projPoints  ESPN_Points  TRUE_Points
2              QB  19.219091   17.836364    17.836364    17.408078
5              WR  11.259677   13.167258    13.167258    12.745610
3              RB  10.121818   11.612000    11.612000    10.980963
1               K   9.571429    8.261429     8.261429     8.261429
4              TE   9.540000   10.539000    10.539000    10.305258
0            D/ST   5.857143    6.084286     6.084286     6.252983

In [354]:
# Box plot for 'points'
plt.figure(figsize=(10, 6))
sns.boxplot(x='primaryPosition', y='points', data=LINEUPS[(LINEUPS['player_active_status'] == 'active') & (~LINEUPS['primaryPosition'].isin(['DE', 'DT', 'CB', 'S', 'LB'])) & (LINEUPS['points'] > 1)  & (LINEUPS['week'] < 2)])
plt.title('Box Plot of Points by Primary Position')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

# Box plot for 'TRUE_Points'
plt.figure(figsize=(10, 6))
sns.boxplot(x='primaryPosition', y='ESPN_Points', data=LINEUPS[(LINEUPS['player_active_status'] == 'active') & (~LINEUPS['primaryPosition'].isin(['DE', 'DT', 'CB', 'S', 'LB'])) & (LINEUPS['points'] > 1) & (LINEUPS['week'] < 2)])
plt.title('Box Plot of TRUE Points by Primary Position')
plt.xticks(rotation=45)
plt.show()

In [367]:
def plot_proj(df = LINEUPS, week=curr_week, pos = ['QB', 'RB', 'WR', 'TE']):

    # Filter by Position
    df = df[df['primaryPosition'].isin(pos)]

    # Filter For only Players You'd. consider playing
    df = df[df['TRUE_Points'] > 7]

    df['projDiff'] = df['TRUE_Points'] - df['projPoints']

    df = df[(df['projPoints'] > 0) & (df['week'] == week)].groupby(['primaryPosition', 'player_name'])[['points', 'projPoints', 'TRUE_Points', 'projDiff']].mean().sort_values(['projDiff'], ascending=True)
    df = df.reset_index()

    # Get the top 10 highest and 10 lowest values of projDiff
    top_10_projDiff = df[~df['primaryPosition'].isin(['CB', 'S', 'LB', 'DE', 'DT'])].nlargest(10, 'projDiff')
    bottom_10_projDiff = df[~df['primaryPosition'].isin(['CB', 'S', 'LB', 'DE', 'DT'])].nsmallest(10, 'projDiff')

    # Combine the top and bottom values into one DataFrame
    top_bottom_projDiff = pd.concat([bottom_10_projDiff, top_10_projDiff])

    print(top_10_projDiff)
    print(bottom_10_projDiff)


    plt.figure(figsize=(12, 8))

    scatter_plot = sns.scatterplot(
        y='TRUE_Points', 
        x='projPoints', 
        hue='primaryPosition',  # Color by primaryPosition
        data=df, 
        s=100  # Adjust point size if needed
    )

    for line in range(0, top_bottom_projDiff.shape[0]):
        x_point = top_bottom_projDiff.projPoints.iloc[line]
        y_point = top_bottom_projDiff.TRUE_Points.iloc[line]

        # Create different offsets for different positions to avoid overlap
        # You can adjust these multipliers to spread labels further
        angle = np.pi * line / len(top_bottom_projDiff) * 2  # Distribute around circle
        x_offset = 3 * np.cos(angle)  # Larger offset
        y_offset = 3 * np.sin(angle)

        scatter_plot.annotate(
            top_bottom_projDiff.player_name.iloc[line],
            xy=(x_point, y_point),  # Point location
            xytext=(x_point + x_offset, y_point + y_offset),  # Text location
            fontsize=10,
            color='black',
            ha='center',
            va='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'),
            arrowprops=dict(arrowstyle='-', color='gray', lw=1, alpha=0.7)
        )

    # Add a line with slope=1 and intercept=0 (line of equality)
    #max_val = max(df['TRUE_Points'].max(), df['projPoints'].max())  # Find the maximum value for the plot limit

    # Find the overall min and max across both variables
    overall_min = min(df['projPoints'].min(), df['TRUE_Points'].min())
    overall_max = max(df['projPoints'].max(), df['TRUE_Points'].max())

    # Add padding
    padding = (overall_max - overall_min) * 0.05

    # Set equal limits for both axes
    plt.xlim(overall_min - padding, overall_max + padding)
    plt.ylim(overall_min - padding, overall_max + padding)

    # Also update your diagonal line
    plt.plot([overall_min - padding, overall_max + padding], 
             [overall_min - padding, overall_max + padding], 
             color='red', linestyle='--', label='y = x')
    
    # Calculate position for labels (roughly 1/4 and 3/4 along the diagonal)
    mid_point = (overall_min + overall_max) / 2
    label_offset = (overall_max - overall_min) * 0.30
    
    # "ESPN Projects Higher" label (below diagonal line, in red)
    plt.text(mid_point + label_offset, mid_point - label_offset, 
             'ESPN Projects Higher', 
             fontsize=12, 
             color='red', 
             fontweight='bold',
             ha='center',
             rotation=0,  # Rotate to match the diagonal
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    # "Sportsbook Projects Higher" label (above diagonal line, in blue or green)
    plt.text(mid_point - label_offset, mid_point + label_offset, 
             'Sportsbook Projects Higher', 
             fontsize=12, 
             color='green',  # or 'green' if you prefer
             fontweight='bold',
             ha='center',
             rotation=0,  # Rotate to match the diagonal
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

    # Set plot title and labels
    plt.title('Tommy Projected Points vs. ESPN Projected Points')
    plt.ylabel('Tommy Projected Points')
    plt.xlabel('ESPN Projected Points')

    # Show the plot
    plt.show()

plot_proj(LINEUPS, curr_week, pos = ['RB'])

   primaryPosition             player_name  points  projPoints  TRUE_Points  \
40              RB     Christian McCaffrey     0.0       20.82    21.674978   
39              RB  Jacory Croskey-Merritt     0.0       11.29    12.110310   
38              RB              Nick Chubb     0.0       11.92    12.711821   
37              RB            J.K. Dobbins     0.0       11.77    12.552504   
36              RB            Jordan Mason     0.0       14.67    15.234625   
35              RB          Saquon Barkley     0.0       18.62    19.136615   
34              RB         Omarion Hampton     0.0       12.21    12.637424   
33              RB        Quinshon Judkins     0.0        9.00     9.422895   
32              RB      Travis Etienne Jr.     0.0       12.73    13.138712   
31              RB            James Conner     0.0       13.43    13.832961   

    projDiff  
40  0.854978  
39  0.820310  
38  0.791821  
37  0.782504  
36  0.564625  
35  0.516615  
34  0.427424  
33  0.4228

### Simulation Sandbox

In [368]:
sim = django_simulation(league=league, n_simulations=500)

# Convert each list to a DataFrame
df_general = pd.DataFrame(sim[0])
df_position_odds = pd.DataFrame(sim[1])
df_standings = pd.DataFrame(sim[2])

# Merge DataFrames on 'team' and 'owner' columns (assuming they are common keys)
df_merged = df_general.merge(df_position_odds, on=['team', 'owner'], suffixes=('_general', '_odds'))
df_final = df_merged.merge(df_standings, on=['team', 'owner'], suffixes=('', '_standings'))

print(df_final.sort_values(by=['projected_wins'], ascending=[False]))

playoff_odds_swing(league = league, week = 2)

# Power Rankings
print(pd.DataFrame(django_power_rankings(league=league, week=1)))

Simulating from week 3 to 14


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 422 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.6s finished


                      team              owner  projected_wins  \
0         Senorita Awesome     Tommy Winfield            13.1   
1             PetroDragons    Bryan Stieneker            10.8   
2             Tony P's Hog     Turner Banwell            10.7   
3     She De On my Sanders  Charlie Voudouris            10.5   
9          Kyle Pitts Year      Walker Thomas             8.8   
5                   Room40    Thomas Freeland             6.4   
4   Atlanta Dream Superfan         Sam Boxley             6.2   
6                Team Ryan       Patrick Ryan             5.5   
10           The Fireballz           Corson P             5.0   
11           Team Crenshaw    Miller Crenshaw             4.7   
13  Skrilla Aura King Dart     Robert Cooleen             4.7   
7       Waddle These Knuts      Peter Knutson             4.4   
8       Tacos y enchiladas       Andrew Blair             4.4   
12   Cantrell Underdog Men      Alex Cantrell             2.8   

    projected_losses  pr

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished


Simulating from week 3 to 14


...

Simulating from week 3 to 14


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  73 out of 100 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished


Simulating from week 3 to 14


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished


                      team  value              owner
0             PetroDragons  21.75    Bryan Stieneker
1         Senorita Awesome  20.95     Tommy Winfield
2   Atlanta Dream Superfan  20.00         Sam Boxley
3     She De On my Sanders  18.60  Charlie Voudouris
4          Kyle Pitts Year  17.95      Walker Thomas
5             Tony P's Hog  17.65     Turner Banwell
6   Skrilla Aura King Dart  17.15     Robert Cooleen
7            Team Crenshaw  16.25    Miller Crenshaw
8       Tacos y enchiladas  15.60       Andrew Blair
9       Waddle These Knuts  14.55      Peter Knutson
10           The Fireballz  14.30           Corson P
11   Cantrell Underdog Men  13.40      Alex Cantrell
12                  Room40  11.70    Thomas Freeland
13               Team Ryan  10.75       Patrick Ryan


### Historical Stats

- **Efficiency**
    - The rate at which a team starts an optimal lineup

In [319]:
league_history = scrape_team_stats(league_id=lg_vars[select_league]['ID'],
        start_year=2022,
        end_year=lg_vars[select_league]['end'],
        swid=lg_vars[select_league]['SWID'],
        espn_s2=lg_vars[select_league]['ESPN_S2'])

league_history = league_history[(league_history['year'] <= 2024) | ((league_history['year'] == 2025) & (league_history['week'] <= curr_week))]

weekly_stats_analysis(df=league_history, year=2025, week=curr_week - 1)
print(" ")
season_stats_analysis(league=league, df=league_history, week=curr_week - 1)
print(" ")
all_time_stats_analysis(league=league, df=league_history)


[BUILDING LEAGUE] Fetching historical stats for 2022...
BUILDING 2022 Season For Knights FFL  

[BUILDING LEAGUE] Fetching historical stats for 2023...
BUILDING 2023 Season For Knights FFL  

[BUILDING LEAGUE] Fetching historical stats for 2024...
BUILDING 2024 Season For Knights FFL  

[BUILDING LEAGUE] Fetching historical stats for 2025...
BUILDING 2025 Season For Knights FFL  
----------------------------------------------------------------
|                        Week  2 Analysis                      |
----------------------------------------------------------------
League-wide POSITIVE stats
--------------------------
Peter Knutson had the 3rd highest K_pts (23.00 pts) in league history


Franchise POSITIVE stats
--------------------------
Tommy Winfield had the 3rd highest team_score (150.10 pts) in franchise history
Thomas Freeland had the 2nd highest team_score_adj (141.96 pts) in franchise history
Tommy Winfield had the 1st highest team_score_adj (172.76 pts) in franchise hi

In [369]:
h2h_build(df = league_history, owner1 = 'Tommy Winfield', owner2='Corson P')

games  wins  losses  ties         PF         PA  \
team_owner     opp_owner                                                    
Tommy Winfield Corson P       3     1       2     0  94.793333  98.273333   

                          margin   win_pct  
team_owner     opp_owner                    
Tommy Winfield Corson P    -3.48  0.333333

In [ ]:
years = range_values = list(range(2024, 2025))
weeks = range_values = list(range(1, 17))
lab = f"{select_league} - Historical Stats From {years[0]} to {years[-1]}"

team_stats_list = []
for y in years:
    league = fetch_league(
        league_id=lg_vars[select_league]['ID'],
        year=y,
        swid=lg_vars[select_league]['SWID'],
        espn_s2=lg_vars[select_league]['ESPN_S2']
    )
    for week in weeks:
        for i in league.teams:
            # Init DF
            team_stats = {}

            # Teams
            team_lineup = get_lineup(league, i, week=week)
            opp_lineup = get_opp_lineup(league, i, week=week)

            # League Information
            team_stats['leagueName'] = league.name

            # Time
            team_stats['year'] = y
            team_stats['week'] = week

            # Team Information
            team_stats['teamID'] = i.team_id
            team_stats['teamOwner'] = i.owner


            # Efficiency Stats
            team_stats['trueScore'] = np.sum([player.points for player in team_lineup if player.slot_position not in ("BE", "IR")])
            team_stats['bestScore'] = get_best_lineup(league, team_lineup)
            team_stats['efficiencyScore'] = get_lineup_efficiency(league, team_lineup)
            team_stats['projAdherance'] = get_idiot_score(league, team_lineup)
            team_stats['scoreDifference'] = get_score_surprise(league, team_lineup)

            # OppStats
            team_stats['opp_trueScore'] = np.sum([player.points for player in opp_lineup if player.slot_position not in ("BE", "IR")])
            team_stats['opp_bestScore'] = get_best_lineup(league, opp_lineup)
            team_stats['opp_efficiencyScore'] = get_lineup_efficiency(league, opp_lineup)
            team_stats['opp_projAdherance'] = get_idiot_score(league, opp_lineup)
            team_stats['opp_scoreDifference'] = get_score_surprise(league, opp_lineup)

            # Differences
            team_stats['margin'] = team_stats['trueScore'] - team_stats['opp_trueScore']
            team_stats["optimalMargin"] = team_stats['bestScore'] - team_stats['opp_bestScore']
            team_stats["efficiencyMargin"] = team_stats['efficiencyScore'] - team_stats['opp_efficiencyScore']
            team_stats["scoreDifferenceMargin"] = team_stats['scoreDifference'] - team_stats['opp_scoreDifference']

            # Outcomes
            team_stats['isWin'] = np.where(team_stats['margin'] > 0, 1, 0)
            team_stats['isTie'] = np.where(team_stats['margin'] == 0, 1, 0)
            team_stats['isLoss'] = np.where(team_stats['margin'] < 0, 1, 0)
            team_stats['isOptimalWin'] = np.where(team_stats['optimalMargin'] > 0, 1, 0)
            team_stats['isOptimalTie'] = np.where(team_stats['optimalMargin'] == 0, 1, 0)
            team_stats['isOptimalLoss'] = np.where(team_stats['optimalMargin'] < 0, 1, 0)
            team_stats['netWinsAdded'] = np.where((team_stats['isWin'] == 1) & (team_stats['isOptimalWin'] == 0), 1, 0)
            team_stats['netLossAdded'] = np.where((team_stats['isLoss'] == 1) & (team_stats['isOptimalWin'] == 1), 1, 0)
            team_stats['netLuck'] = team_stats['netWinsAdded'] - team_stats['netLossAdded']

            # Luck
            try:
                team_stats['luckIndex'] = get_weekly_luck_index(league=league, team = i, week=week)
            except:
                team_stats['luckIndex'] = 0

            # Append
            if y < 2024 or (y == 2024 and week < 3):
                team_stats_list.append(team_stats)
                
team_stats_df = pd.DataFrame(team_stats_list)


In [ ]:
# Group By # , 'year'
hist_stats_df = team_stats_df.groupby(['leagueName', 'week', 'teamID', 'teamOwner']).agg(
    Efficency = ('efficiencyScore', 'mean'),
    W=("isWin", "sum"),
    L=("isLoss", "sum"),
    T=("isTie", "sum"),
    xW=("isOptimalWin", "sum"),
    xL=("isOptimalLoss", "sum"),
    xT=("isOptimalTie", "sum"),
    Luck_W=("netLuck", "sum"),
    TruePF=('trueScore', 'mean'),
    OptimalPF=('bestScore', 'mean'),
    TruePA=('opp_trueScore', 'mean'),
    OptimalPA=('opp_bestScore', 'mean'),
    AdhereProj = ('projAdherance', 'mean'),
    TrueMargin = ('margin', 'mean'),
    #OptimalMargin = ('optimalMargin', 'mean'),
    #PF_OnTable = ('scoreDifference', 'mean'),
    Luck_Index = ('luckIndex', 'mean')
    ).reset_index()\
     .sort_values(by=['Luck_Index'], ascending=False)

print(lab)
hist_stats_df

In [ ]:
# Mapping
years = hist_stats_df['year'].sort_values(ascending=False).unique()
cmap = plt.get_cmap('tab10')
color_dict = {year: cmap(i) for i, year in enumerate(years)}

# Plotting
plt.figure(figsize=(12, 8))

plt.scatter(
    hist_stats_df['AdhereProj'],
    hist_stats_df['Efficency'],
    c=hist_stats_df['year'].map(color_dict),
    label='Data Points')

# Adding labels
texts = []
for i in range(len(hist_stats_df)):
    owner_year = f"{hist_stats_df['teamOwner'][i]}"
    texts.append(plt.text(hist_stats_df['AdhereProj'][i], hist_stats_df['Efficency'][i],
                          owner_year, fontsize=9, ha='center'))
    
adjust_text(texts, arrowprops=dict(arrowstyle="->", color='red', lw=0.5))

handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_dict[year], markersize=10) 
           for year in years]
plt.legend(handles, years, title="Year")
    

plt.xlabel('Adherance To ESPN Projections')
plt.ylabel('Correct Start Rate')
plt.title('Scatter Plot of Adherance To Projections vs. Correct Start Rate')
plt.grid(True)
plt.show()

## Projections

In [ ]:
years = range_values = list(range(2024, 2025))
stats_df = pd.DataFrame()
for y in years:
        ply_stats = get_stats_by_matchup(league_id=lg_vars[select_league]['ID'],
                year=y,
                swid=lg_vars[select_league]['SWID'],
                espn_s2=lg_vars[select_league]['ESPN_S2'])
        
        stats_df = pd.concat([stats_df, ply_stats]).fillna(0)


stats_df.head()

In [ ]:
stats_df['diff'] = stats_df['points'] - stats_df['projPoints']
# Set up plot area
plt.figure(figsize=(12, 8))

# Create a violin plot for each primaryPosition
sns.violinplot(x='slotPosition', y='points', data=stats_df)

# Labels and title
plt.xlabel('Primary Position')
plt.ylabel('Points')
plt.title('Distribution of Points by Primary Position')

# Show plot
plt.show()

In [ ]:
stats_df = stats_df[stats_df['week'] < 3]
grouped_df = stats_df[stats_df['week'] < 3].groupby(['year','week', 'primaryPosition', 'player_name'])

positions = stats_df['primaryPosition'].unique()

# Set up plot area
plt.figure(figsize=(20, 12))

# Loop through each primaryPosition
for pos in positions:
    pos_data = stats_df[stats_df['primaryPosition'] == pos]
    
    # Scatter plot
    plt.scatter(pos_data['points'], pos_data['projPoints'], label=pos)

    # Calculate R² using Linear Regression
    X = pos_data['points'].values.reshape(-1, 1)
    y = pos_data['projPoints'].values
    model = LinearRegression().fit(X, y)
    r2 = r2_score(y, model.predict(X))

    # Display R² value
    plt.text(pos_data['points'].mean(), pos_data['projPoints'].mean(), f"R²: {r2:.2f}", fontsize=9)

# Labels and legend
plt.xlabel('Points')
plt.ylabel('Projected Points')
plt.title('Points vs Projected Points by Primary Position')
plt.legend(title="Primary Position")
plt.show()

In [ ]:
# Get unique positions
positions = stats_df['player_name'].unique()

# Set up multi-plot grid
n_cols = 3
n_rows = (len(positions) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))
axes = axes.flatten()

# Loop through each player_name and create scatter plots
for i, pos in enumerate(positions):
    pos_data = stats_df[stats_df['player_name'] == pos]
    
    # Scatter plot
    sns.scatterplot(x='points', y='projPoints', data=pos_data, ax=axes[i], color='blue')
    
    # Calculate R² using Linear Regression
    X = pos_data['points'].values.reshape(-1, 1)
    y = pos_data['projPoints'].values
    model = LinearRegression().fit(X, y)
    r2 = r2_score(y, model.predict(X))
    
    # Add R² value to the plot
    axes[i].text(0.05, 0.95, f'R²: {r2:.2f}', transform=axes[i].transAxes, fontsize=12, verticalalignment='top')
    
    # Set title and labels
    axes[i].set_title(f"Primary Position: {pos}")
    axes[i].set_xlabel('Points')
    axes[i].set_ylabel('Projected Points')

# Remove empty subplots if any
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout and show plot
plt.tight_layout()
plt.show()

### Positional Variance Between Projections and Actual

### Power Rankings

In [ ]:
league.power_rankings(league.current_week)

In [ ]:
get_season_luck_indices(league, week)